In [12]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Configuración del servidor Flask
app = Flask(__name__)

# Habilitar CORS
CORS(app)

# Configura el URI del servidor MLflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")  # Cambia según tu servidor MLflow
model_name = "Tiempo"  # Nombre del modelo registrado
model_version = "14"  # Versión del modelo en MLflow

# Cargar el modelo desde el registro de MLflow
try:
    model_uri = f"models:/{model_name}/{model_version}"
    print(f"Cargando el modelo desde {model_uri}...")
    model = mlflow.sklearn.load_model(model_uri)
    print("Modelo cargado exitosamente.")
    scaler = StandardScaler()
except Exception as e:
    print(f"Error al cargar el modelo: {e}")
    model = None

@app.route("/", methods=["GET"])
def home():
    return jsonify({"message": "API para estimar tiempo de entrega funcionando correctamente."})

@app.route('/predict', methods=['POST'])
def predict():
    try:
        if model is None:
            return jsonify({'error': 'Modelo no cargado.'}), 500

        # Obtener datos en formato JSON
        data = request.get_json()

        # Validar formato
        if not data or not isinstance(data, list):
            return jsonify({'error': 'Se esperaba una lista de instancias JSON.'}), 400

        # Crear DataFrame con los datos recibidos
        input_data = pd.DataFrame(data)

        # Validar que las columnas requeridas estén presentes
        required_columns = ["distancia_km", "numero_articulos", "velocidad_kmh"]
        if not all(col in input_data.columns for col in required_columns):
            return jsonify({'error': f'Faltan columnas requeridas: {required_columns}'}), 400

        # Preprocesar los datos (escalado)
        input_scaled = scaler.fit_transform(input_data)

        # Realizar predicciones
        predictions = model.predict(input_scaled)

        return jsonify({'predictions': predictions.tolist()})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(host='127.0.0.1', port=5001, debug=True, use_reloader=False)


Cargando el modelo desde models:/Tiempo/14...
Modelo cargado exitosamente.
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [20/Jan/2025 22:32:17] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2025 22:32:17] "POST /predict HTTP/1.1" 400 -
127.0.0.1 - - [20/Jan/2025 22:37:00] "OPTIONS /predict HTTP/1.1" 200 -
C:\Users\dgldr\anaconda3\envs\envname\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
127.0.0.1 - - [20/Jan/2025 22:37:00] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2025 22:39:11] "OPTIONS /predict HTTP/1.1" 200 -
C:\Users\dgldr\anaconda3\envs\envname\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
127.0.0.1 - - [20/Jan/2025 22:39:11] "POST /predict HTTP/1.1" 200 -
